In [1]:
import pandas as pd
import my_utils as mu
import requests

In [2]:
raw_data = mu.get_rawdata('DIAMOND')

100%|██████████| 12/12 [01:13<00:00,  6.11s/it]


In [3]:
tmp_list = list(map(lambda x : x['events'], raw_data.iloc[1].timelines['info']['frames']))

In [4]:
tmp_list

[[{'realTimestamp': 1679628190829, 'timestamp': 0, 'type': 'PAUSE_END'}],
 [{'itemId': 1036,
   'participantId': 4,
   'timestamp': 5900,
   'type': 'ITEM_PURCHASED'},
  {'itemId': 2003,
   'participantId': 4,
   'timestamp': 6527,
   'type': 'ITEM_PURCHASED'},
  {'itemId': 2003,
   'participantId': 4,
   'timestamp': 6692,
   'type': 'ITEM_PURCHASED'},
  {'itemId': 2003,
   'participantId': 4,
   'timestamp': 7287,
   'type': 'ITEM_PURCHASED'},
  {'itemId': 3340,
   'participantId': 4,
   'timestamp': 7716,
   'type': 'ITEM_PURCHASED'},
  {'itemId': 1054,
   'participantId': 1,
   'timestamp': 9203,
   'type': 'ITEM_PURCHASED'},
  {'itemId': 1056,
   'participantId': 8,
   'timestamp': 10293,
   'type': 'ITEM_PURCHASED'},
  {'afterId': 0,
   'beforeId': 1054,
   'goldGain': 450,
   'participantId': 1,
   'timestamp': 10392,
   'type': 'ITEM_UNDO'},
  {'itemId': 2003,
   'participantId': 8,
   'timestamp': 10656,
   'type': 'ITEM_PURCHASED'},
  {'itemId': 2003,
   'participantId': 8,
 

In [6]:
event_list = [element for array in tmp_list for element in array]

In [30]:
event_list

[{'realTimestamp': 1679628190829, 'timestamp': 0, 'type': 'PAUSE_END'},
 {'itemId': 1036,
  'participantId': 4,
  'timestamp': 5900,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 2003,
  'participantId': 4,
  'timestamp': 6527,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 2003,
  'participantId': 4,
  'timestamp': 6692,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 2003,
  'participantId': 4,
  'timestamp': 7287,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 3340,
  'participantId': 4,
  'timestamp': 7716,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 1054,
  'participantId': 1,
  'timestamp': 9203,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 1056,
  'participantId': 8,
  'timestamp': 10293,
  'type': 'ITEM_PURCHASED'},
 {'afterId': 0,
  'beforeId': 1054,
  'goldGain': 450,
  'participantId': 1,
  'timestamp': 10392,
  'type': 'ITEM_UNDO'},
 {'itemId': 2003,
  'participantId': 8,
  'timestamp': 10656,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 2003,
  'participantId': 8,
  'timestamp': 10854,
  'type': 'ITEM_PURCH

In [9]:
def assi_calc(x):
    try:
        return ','.join(list(map(lambda y: str(y), x['assistingParticipantIds'])))
    except:
        return ''

In [11]:
tower_log = list(filter(lambda x : x['type'] == 'BUILDING_KILL', event_list))

In [16]:
tower_log

[{'bounty': 0,
  'buildingType': 'TOWER_BUILDING',
  'killerId': 6,
  'laneType': 'TOP_LANE',
  'position': {'x': 981, 'y': 10441},
  'teamId': 100,
  'timestamp': 674129,
  'towerType': 'OUTER_TURRET',
  'type': 'BUILDING_KILL'},
 {'bounty': 0,
  'buildingType': 'TOWER_BUILDING',
  'killerId': 0,
  'laneType': 'BOT_LANE',
  'position': {'x': 10504, 'y': 1029},
  'teamId': 100,
  'timestamp': 680602,
  'towerType': 'OUTER_TURRET',
  'type': 'BUILDING_KILL'},
 {'bounty': 0,
  'buildingType': 'TOWER_BUILDING',
  'killerId': 8,
  'laneType': 'MID_LANE',
  'position': {'x': 5846, 'y': 6396},
  'teamId': 100,
  'timestamp': 699826,
  'towerType': 'OUTER_TURRET',
  'type': 'BUILDING_KILL'},
 {'bounty': 0,
  'buildingType': 'TOWER_BUILDING',
  'killerId': 6,
  'laneType': 'TOP_LANE',
  'position': {'x': 1512, 'y': 6699},
  'teamId': 100,
  'timestamp': 725619,
  'towerType': 'INNER_TURRET',
  'type': 'BUILDING_KILL'},
 {'assistingParticipantIds': [9],
  'bounty': 0,
  'buildingType': 'TOWER_B

In [29]:
# laneTower ==> 'laneType' == teamposition(똑같지 않고 특정 방법을 통해서 인식) & 'teamId' == teamId(첫번째 타워 깨진 것만 데이터 입력)
# firstDt -> 최대 1개만 들어감
# laneTower => 타워가 깨졌을 시 1이 들어감(top, mid, bot에만 데이터가 1이 들어가야 함)
# laneTowerTime => 깨진 시간(timestamp) 넣으면 됨.
# 데이터프레임 tmp_lst에 추가

In [82]:
df_create = []
for i in range(len(raw_data)) :
    matchId = raw_data.iloc[i].match_id
    match = raw_data.iloc[i].matches['info']
    timeline = raw_data.iloc[i].timelines['info']

    tower_log = list(map(lambda x : list(filter(lambda z : z['type'] == 'BUILDING_KILL', x['events'])) , timeline['frames']))
    tower_log = [element for array in tower_log for element in array]

    top_b = []
    top_r = []
    mid_b = []
    mid_r = []
    bot_b = []
    bot_r = []
    jungle = []
    sup = []

    for j in range(len(tower_log)) :
        if tower_log[j]['teamId'] == 100 :
            if str(tower_log[j]['laneType']).split('_')[0] == 'TOP' :
                participantId = tower_log[j]['killerId']
                teamId = tower_log[j]['teamId']
                if j == 0 :
                    firstDT = 1
                else :
                    firstDT = 0
                laneTower = 1
                LaneTowerTime = tower_log[j]['timestamp']
                top_b = [participantId, teamId, firstDT, laneTower, LaneTowerTime]
            elif str(tower_log[j]['laneType']).split('_')[0] == 'MID' :
                participantId = tower_log[j]['killerId']
                teamId = tower_log[j]['teamId']
                if j == 0 :
                    firstDT = 1
                else :
                    firstDT = 0
                laneTower = 1
                LaneTowerTime = tower_log[j]['timestamp']
                mid_b = [participantId, teamId, firstDT, laneTower, LaneTowerTime]
            elif str(tower_log[j]['laneType']).split('_')[0] == 'BOT' :
                participantId = tower_log[j]['killerId']
                teamId = tower_log[j]['teamId']
                if j == 0 :
                    firstDT = 1
                else :
                    firstDT = 0
                laneTower = 1
                LaneTowerTime = tower_log[j]['timestamp']
                bot_b = [participantId, teamId, firstDT, laneTower, LaneTowerTime]
        elif tower_log[j]['teamId'] == 200 :
            if str(tower_log[j]['laneType']).split('_')[0] == 'TOP' :
                participantId = tower_log[j]['killerId']
                teamId = tower_log[j]['teamId']
                if j == 0 :
                    firstDT = 1
                else :
                    firstDT = 0
                laneTower = 1
                LaneTowerTime = tower_log[j]['timestamp']
                top_r = [participantId, teamId, firstDT, laneTower, LaneTowerTime]
            elif str(tower_log[j]['laneType']).split('_')[0] == 'MID' :
                participantId = tower_log[j]['killerId']
                teamId = tower_log[j]['teamId']
                if j == 0 :
                    firstDT = 1
                else :
                    firstDT = 0
                laneTower = 1
                LaneTowerTime = tower_log[j]['timestamp']
                mid_r = [participantId, teamId, firstDT, laneTower, LaneTowerTime]
            elif str(tower_log[j]['laneType']).split('_')[0] == 'BOT' :
                participantId = tower_log[j]['killerId']
                teamId = tower_log[j]['teamId']
                if j == 0 :
                    firstDT = 1
                else :
                    firstDT = 0
                laneTower = 1
                LaneTowerTime = tower_log[j]['timestamp']
                bot_r = [participantId, teamId, firstDT, laneTower, LaneTowerTime]

    for j in range(10) :
        if parti

        lst_tmp.append(matchId)
        lst_tmp.append(match['gameDuration'])
        lst_tmp.append(match['gameVersion'])
        participantId = match['participants'][0]['participantId']
        teamId = match['participants'][0]['teamId']

        blue_ban = list(map(lambda x : str(x['championId']), match['teams'][0]['bans']))
        red_ban = list(map(lambda x : str(x['championId']), match['teams'][1]['bans']))
        ban_list = list(set(blue_ban + red_ban))
        lst_tmp.append('|'.join(ban_list))

        kill_log = list(map(lambda x : list(filter(lambda z : z['type'] == 'CHAMPION_KILL', x['events'])) , timeline['frames']))
        kill_log = [element for array in kill_log for element in array]

        k = list(map(lambda x : str(x['killerId']), kill_log))
        v = list(map(lambda x : str(x['victimId']), kill_log))
        a = list(map(lambda x : assi_calc(x), kill_log))

        lst_tmp.append('|'.join(k))
        lst_tmp.append('|'.join(v))
        lst_tmp.append('|'.join(a))

        df_create.append(lst_tmp)

    col_lst = ['matchId', 'gameDuration','gameVersion', 'participantId', 'teamId', 'firstDT', 'laneTower','LaneTowerTime' , 'bans', 'killerId', 'victimId', 'assistId']
    df = pd.DataFrame(df_create, columns = col_lst)

ValueError: 12 columns passed, passed data had 7 columns

In [78]:
tower_log[0]['teamId'] == 100

True

In [80]:
tower_log[0]['killerId']

6

In [14]:
# col_lst = ['matchId','gameDuration','gameVersion', 'participantId', 'teamId', 'firstDT','laneTower','LaneTowerTime' ,'bans','killerId','victimId','assistId']

In [46]:
# removing duplicated from the list using list comprehension

# initializing list
sam_list = list(map(lambda  x : str(x['laneType']).split('_')[0], tower_log))
print ("The list is: " + str(sam_list))


# to remove duplicated from list
result = []
[result.append(x) for x in sam_list if x not in result]

# printing list after removal
print ("The list after removing duplicates: " + str(result))

The list is: ['TOP', 'BOT', 'MID', 'TOP', 'MID', 'BOT', 'MID', 'BOT', 'MID', 'BOT', 'MID', 'MID']
The list after removing duplicates: ['TOP', 'BOT', 'MID']


In [22]:
p = list(map(lambda  x : str(x['killerId']), tower_log))

In [63]:
tower_log[0].keys()

dict_keys(['bounty', 'buildingType', 'killerId', 'laneType', 'position', 'teamId', 'timestamp', 'towerType', 'type'])

In [23]:
p

['6', '0', '8', '6', '8', '6', '9', '6', '9', '8', '6', '9']

In [25]:
towertime = list(map(lambda  x : str(x['timestamp']), tower_log))

In [26]:
towertime

['674129',
 '680602',
 '699826',
 '725619',
 '803079',
 '817349',
 '824879',
 '839776',
 '849684',
 '850081',
 '862468',
 '866795']

In [27]:
towertime = list(map(lambda  x : str(x['teamId']), tower_log))

In [28]:
towertime

['100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100']

In [48]:
tower_log = list(map(lambda x : list(filter(lambda z : z['type'] == 'BUILDING_KILL', x['events'])) , timeline['frames']))
tower_log = [element for array in tower_log for element in array]

In [49]:
tower_log

[{'bounty': 0,
  'buildingType': 'TOWER_BUILDING',
  'killerId': 6,
  'laneType': 'TOP_LANE',
  'position': {'x': 981, 'y': 10441},
  'teamId': 100,
  'timestamp': 739558,
  'towerType': 'OUTER_TURRET',
  'type': 'BUILDING_KILL'},
 {'assistingParticipantIds': [8, 10],
  'bounty': 0,
  'buildingType': 'TOWER_BUILDING',
  'killerId': 7,
  'laneType': 'BOT_LANE',
  'position': {'x': 10504, 'y': 1029},
  'teamId': 100,
  'timestamp': 836731,
  'towerType': 'OUTER_TURRET',
  'type': 'BUILDING_KILL'},
 {'bounty': 0,
  'buildingType': 'TOWER_BUILDING',
  'killerId': 6,
  'laneType': 'MID_LANE',
  'position': {'x': 5846, 'y': 6396},
  'teamId': 100,
  'timestamp': 879867,
  'towerType': 'OUTER_TURRET',
  'type': 'BUILDING_KILL'},
 {'assistingParticipantIds': [8, 10],
  'bounty': 0,
  'buildingType': 'TOWER_BUILDING',
  'killerId': 9,
  'laneType': 'MID_LANE',
  'position': {'x': 5048, 'y': 4812},
  'teamId': 100,
  'timestamp': 1032460,
  'towerType': 'INNER_TURRET',
  'type': 'BUILDING_KILL'}

In [51]:
# removing duplicated from the list using list comprehension

# initializing list
sam_list = list(map(lambda  x : str(x['laneType']).split('_')[0], tower_log))
print ("The list is: " + str(sam_list))


# to remove duplicated from list
result = []
[result.append(x) for x in sam_list if x not in result]

# printing list after removal
print ("The list after removing duplicates: " + str(result))

The list is: ['TOP', 'BOT', 'MID', 'MID', 'BOT', 'TOP', 'MID', 'MID', 'MID', 'MID']
The list after removing duplicates: ['TOP', 'BOT', 'MID']


In [70]:
raw_data.iloc[i].matches['info']['participants'][0]['teamPosition']

'TOP'

In [66]:
raw_data.iloc[i].matches['info']['participants'][0]['participantId']

1

In [75]:
str(tower_log[0]['laneType']).split('_')[0]

'TOP'

In [ ]:
df_create = []
for i in range(len(tmp)):
    matchId = tmp.iloc[i].match_id
    match = tmp.iloc[i].matches['info']
    timeline = tmp.iloc[i].timeline['info']
    for j in range(10):
        lst_tmp = []
        lst_tmp.append(matchId)
        lst_tmp.append(match['gameDuration'])
        lst_tmp.append(match['gameVersion'])

        blue_ban = list(map(lambda x: str(x['championId']),match['teams'][0]['bans']))
        red_ban = list(map(lambda x: str(x['championId']),match['teams'][1]['bans']))
        ban_list = list(set(blue_ban + red_ban))
        lst_tmp.append('|'.join(ban_list))

        kill_log = list(map(lambda x : list(filter(lambda z: z['type']=='CHAMPION_KILL' ,x['events'])) ,timeline['frames']))
        kill_log = [element for array in kill_log for element in array]
        k = list(map(lambda x:str(x['killerId']) ,kill_log))
        v = list(map(lambda x:str(x['victimId']) ,kill_log))
        a = list(map(lambda x:assi_calc(x) ,kill_log))

        lst_tmp.append('|'.join(k))
        lst_tmp.append('|'.join(v))
        lst_tmp.append('|'.join(a))

        df_create.append(lst_tmp)

col_lst = ['matchId','gameDuration','gameVersion','bans','killerId','victimId','assistId']
df = pd.DataFrame(df_create,columns = col_lst)